# Practice with Beautiful Soup and ETL

Construct a series of cells that will pull this URL and ingest it into SQLite3 database

https://factfinder.census.gov/bkmk/table/1.0/en/GEP/2014/00A4/0100000US

Because the table is loaded via scripts, you'll want to use the following for parsing:
https://indigo.sgn.missouri.edu/static/mirror_sites/factfinder.census.gov/bkmk/table/1.0/en/GEP/2014/00A4/0100000US


![Census Data](../images/census_data_look.png)


### Data table element
```
<table id="data" class="stat-tbl">
```

**Please organize your code in the cells below.** 

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import os
import sqlite3
import numpy
# Pull the HTML link into a local file or buffer
#   and then parse with the BeautifulSoup library
# ------------------------------------------------

# Grab the website data
# Open the URL in your browser and follow along with the page inspector (Ctrl+Shift+I)
url = 'https://indigo.sgn.missouri.edu/static/mirror_sites/factfinder.census.gov/bkmk/table/1.0/en/GEP/2014/00A4/0100000US.html'
r = requests.get(url)

print('Status: ' + str(r.status_code))

soup = BeautifulSoup(r.content, "html")

# Scrape out the table
table = soup.find(id='data')

#print(table.prettify())

# Scrape header for names
header_row = table.thead.findAll('th')
#print(header_row)

# Parse the header names

# First one doesn't have a link with a code name. Rude
readable_names = ['Geographic Area Name']
db_names = ['areaname']

for column in header_row[1:]:
  readable_names.append(column.text)
  messy_name = column.a.attrs['data-vo-id']
  messy_name = messy_name[messy_name.rfind('~') + 1:]
  messy_name = messy_name[messy_name.rfind('.') + 1:].lower()
  db_names.append(messy_name)

name_mapping = dict(zip(db_names, readable_names))
pprint(name_mapping)



In [ ]:
# Convert the data into a list of dictionaries
#   or some other structure you can convert into
#   pandas Data Frame
# ------------------------------------------------
scraped_data = []

# Quick and dirty conversion of types
convert = [str,int,str,int,float,int,float,int,float,int,float,int,float,int,float,int,float,int,float]

for row in table.tbody.findAll('tr'):
  row_data = [x.text for x in row.children]
  for idx in range(0,len(convert)):
      row_data[idx] = convert[idx](row_data[idx].replace(',','')) # ugh commas break int conversion
      # This would kill ANY commas in the data, though. Thankfully there are none.
      # If there were, just gate on the conversion function being int
  scraped_data.append(dict(zip(db_names, row_data)))

pprint(scraped_data[0])

frame = pd.DataFrame(scraped_data)

print(frame.head())

In [ ]:
# Save data into two formats: csv and json
# ------------------------------------------------


frame.to_csv("census.csv", index=False)  # index=False will make sure the row indices (i.e 0, 1, ...) will not 
# written to csv file

frame.to_json("census.json")


# check whether write was done correctly (optional)

df_csv = pd.read_csv("census.csv", header=0)
print(df_csv.head())

print("-" * 80)

df_json = pd.read_json("census.json")
print(df_json.head())


# Save your notebook